In [14]:
from run_train import create_model_and_diffusion
from utils.step_sample import create_named_schedule_sampler
from train import TrainLoop
from utils.data import load_data_text
from tokenizer import load_tokenizer, load_model_emb
from sampling import sampling

from transformers import AutoTokenizer, PreTrainedTokenizerFast, BertTokenizerFast, set_seed
import json, torch, os
from utils import dist_util
from functools import partial
import pickle
import random
from datetime import datetime

In [15]:
dist_util.clear_cache()
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [16]:
lr=0.0001
batch_size=10
microbatch=5
epochs=20_000
eval_interval=100
ema_rate='0.9999' 
schedule_sampler='uniform'
diffusion_steps=1_200
noise_schedule='sqrt'
vocab='custom'
use_plm_init='no' # embedding in transformer
vocab_size=0
config_name='bert-base-uncased'
seq_len=128
hidden_t_dim=128
hidden_dim=128
dropout=0.1
seed=102
weight_decay=0.0
predict_xstart=True
rescale_timesteps=True
emb_scale_factor=1.0

In [17]:
cc_data_dir='data/commonsense'
ss_data_dir='data/shakespeare'
ss_small_data_dir='data/mini-shakespeare'
combined_data_dir='data/combined'
combined_small_data_dir='data/combined/small'
regular_data_dir='data'

# set the data directory
data_dir=regular_data_dir

In [18]:
set_seed(seed)

In [19]:
tokenizer = load_tokenizer('shakespeare_plays', config_name)

In [20]:
model_weight, tokenizer = load_model_emb(hidden_dim, tokenizer)

In [21]:
model_weight

Embedding(30267, 128)

In [22]:
## very very important to set this!!!!!
vocab_size = tokenizer.vocab_size
vocab_size

30267

In [23]:
data = load_data_text(
        batch_size=batch_size,
        seq_len=seq_len,
        data_dir=data_dir,
        loaded_vocab=tokenizer,
        model_emb=model_weight # use model's weights as init
    )

val = load_data_text(
        batch_size=batch_size,
        seq_len=seq_len,
        data_dir=data_dir,
        loaded_vocab=tokenizer,
        split='valid',
        model_emb=model_weight # use model's weights as init
    )

############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading form the TRAIN set...
### Data samples...
 ['o hell! what have we here? a carrion death, within whose empty eye there is a written scroll!', 'and his disciples only envy at, ye blew the fire that burns ye now have at ye! enter king,'] ["i'll read the writing. all that glitters is not gold, often have you heard that told", 'frowning on them, takes his seat']
RAM used: 2838.91 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 48627
})
RAM used: 2868.57 MB


Running tokenizer on dataset (num_proc=4):   0%|          | 0/48627 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 48627
})
### tokenized_datasets...example [2, 36, 1299, 5, 163, 149, 132, 236, 21, 22, 7134, 431, 9, 905, 568, 3065, 755, 209, 120, 22, 4179, 7421, 5, 3]
RAM used: 2917.21 MB


merge and mask:   0%|          | 0/48627 [00:00<?, ? examples/s]

RAM used: 2947.92 MB


...

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 12147
})
### tokenized_datasets...example [2, 3885, 120, 104, 519, 9, 2545, 8, 40, 477, 9, 22, 210, 253, 1232, 9839, 186, 4042, 11, 3]
RAM used: 3007.48 MB


merge and mask:   0%|          | 0/12147 [00:00<?, ? examples/s]

RAM used: 3019.97 MB


padding:   0%|          | 0/12147 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 12147
}) padded dataset
RAM used: 3040.57 MB
RAM used: 3040.57 MB


In [24]:
model, diffusion = create_model_and_diffusion(
                        hidden_t_dim,
                        hidden_dim,
                        vocab_size,
                        config_name,
                        use_plm_init,
                        dropout,
                        diffusion_steps,
                        noise_schedule,
                        predict_xstart,
                        rescale_timesteps,
                    )

model.to(dist_util.dev())

TransformerNetModel(
  (word_embedding): Embedding(30267, 128)
  (lm_head): Linear(in_features=128, out_features=30267, bias=True)
  (time_embed): Sequential(
    (0): Linear(in_features=128, out_features=512, bias=True)
    (1): SiLU()
    (2): Linear(in_features=512, out_features=768, bias=True)
  )
  (input_up_proj): Sequential(
    (0): Linear(in_features=128, out_features=768, bias=True)
    (1): Tanh()
    (2): Linear(in_features=768, out_features=768, bias=True)
  )
  (input_transformers): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768,

In [25]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

91192379

In [ ]:
schedule_sampler = create_named_schedule_sampler('uniform', diffusion)

TrainLoop(
        model=model,
        diffusion=diffusion,
        data=data,
        batch_size=batch_size,
        microbatch=microbatch,
        lr=lr,
        ema_rate=ema_rate,
        schedule_sampler=schedule_sampler,
        weight_decay=weight_decay,
        epochs=epochs,
        eval_data=val,
        eval_interval=eval_interval
    ).run_loop()



======== Training starts now ========


Epoch 1/20000 Training Loss: 1.043076992034912
Epoch 2/20000 Training Loss: 0.695614218711853
Epoch 3/20000 Training Loss: 0.5766768455505371
Epoch 4/20000 Training Loss: 0.5150974988937378
Epoch 5/20000 Training Loss: 0.5483779907226562
Epoch 6/20000 Training Loss: 0.47229117155075073
Epoch 7/20000 Training Loss: 0.5219022035598755
Epoch 8/20000 Training Loss: 0.5533789396286011
Epoch 9/20000 Training Loss: 0.45337945222854614
Epoch 10/20000 Training Loss: 0.49771273136138916
Epoch 11/20000 Training Loss: 0.5337638854980469
Epoch 12/20000 Training Loss: 0.5432272553443909
Epoch 13/20000 Training Loss: 0.4817114472389221
Epoch 14/20000 Training Loss: 0.5254583358764648
Epoch 15/20000 Training Loss: 0.4739689230918884
Epoch 16/20000 Training Loss: 0.4796767830848694
Epoch 17/20000 Training Loss: 0.45504820346832275
Epoch 18/20000 Training Loss: 0.4943806529045105
Epoch 19/20000 Training Loss: 0.5340105891227722
Epoch 20/20000 Training Loss: 0.46

Epoch 162/20000 Training Loss: 0.30627384781837463
Epoch 163/20000 Training Loss: 0.3177127540111542
Epoch 164/20000 Training Loss: 0.31958019733428955
Epoch 165/20000 Training Loss: 0.3083963394165039
Epoch 166/20000 Training Loss: 0.2914397716522217
Epoch 167/20000 Training Loss: 0.3150452971458435
Epoch 168/20000 Training Loss: 0.28211510181427
Epoch 169/20000 Training Loss: 0.3398022949695587
Epoch 170/20000 Training Loss: 0.2901287078857422
Epoch 171/20000 Training Loss: 0.3351472318172455
Epoch 172/20000 Training Loss: 0.303388386964798
Epoch 173/20000 Training Loss: 0.30401650071144104
Epoch 174/20000 Training Loss: 0.32007336616516113
Epoch 175/20000 Training Loss: 0.27891016006469727
Epoch 176/20000 Training Loss: 0.27343934774398804
Epoch 177/20000 Training Loss: 0.30770373344421387
Epoch 178/20000 Training Loss: 0.329143226146698
Epoch 179/20000 Training Loss: 0.30214378237724304
Epoch 180/20000 Training Loss: 0.29184016585350037
Epoch 181/20000 Training Loss: 0.321536839008

Epoch 317/20000 Training Loss: 0.18392035365104675
Epoch 318/20000 Training Loss: 0.18830201029777527
Epoch 319/20000 Training Loss: 0.21389225125312805
Epoch 320/20000 Training Loss: 0.20672489702701569
Epoch 321/20000 Training Loss: 0.21182310581207275
Epoch 322/20000 Training Loss: 0.17571106553077698
Epoch 323/20000 Training Loss: 0.2023884803056717
Epoch 324/20000 Training Loss: 0.21646863222122192
Epoch 325/20000 Training Loss: 0.1732669174671173
Epoch 326/20000 Training Loss: 0.187344029545784
Epoch 327/20000 Training Loss: 0.22465357184410095
Epoch 328/20000 Training Loss: 0.20943115651607513
Epoch 329/20000 Training Loss: 0.1829974204301834
Epoch 330/20000 Training Loss: 0.1954069435596466
Epoch 331/20000 Training Loss: 0.21412129700183868
Epoch 332/20000 Training Loss: 0.18924666941165924
Epoch 333/20000 Training Loss: 0.18774829804897308
Epoch 334/20000 Training Loss: 0.2153984159231186
Epoch 335/20000 Training Loss: 0.20108318328857422
Epoch 336/20000 Training Loss: 0.19382

Epoch 475/20000 Training Loss: 0.12968255579471588
Epoch 476/20000 Training Loss: 0.12339361011981964
Epoch 477/20000 Training Loss: 0.13884003460407257
Epoch 478/20000 Training Loss: 0.1608664095401764
Epoch 479/20000 Training Loss: 0.1358514279127121
Epoch 480/20000 Training Loss: 0.14105093479156494
Epoch 481/20000 Training Loss: 0.1316950023174286
Epoch 482/20000 Training Loss: 0.14121533930301666
Epoch 483/20000 Training Loss: 0.15275990962982178
Epoch 484/20000 Training Loss: 0.16710670292377472
Epoch 485/20000 Training Loss: 0.1700659543275833
Epoch 486/20000 Training Loss: 0.12480179965496063
Epoch 487/20000 Training Loss: 0.14827370643615723
Epoch 488/20000 Training Loss: 0.13110977411270142
Epoch 489/20000 Training Loss: 0.13292726874351501
Epoch 490/20000 Training Loss: 0.17636848986148834
Epoch 491/20000 Training Loss: 0.17391487956047058
Epoch 492/20000 Training Loss: 0.09788326174020767
Epoch 493/20000 Training Loss: 0.13484010100364685
Epoch 494/20000 Training Loss: 0.14

Epoch 632/20000 Training Loss: 0.1010279655456543
Epoch 633/20000 Training Loss: 0.11987681686878204
Epoch 634/20000 Training Loss: 0.10534965991973877
Epoch 635/20000 Training Loss: 0.14259986579418182
Epoch 636/20000 Training Loss: 0.12641270458698273
Epoch 637/20000 Training Loss: 0.127445250749588
Epoch 638/20000 Training Loss: 0.11220333725214005
Epoch 639/20000 Training Loss: 0.10971663892269135
Epoch 640/20000 Training Loss: 0.09641699492931366
Epoch 641/20000 Training Loss: 0.10650657117366791
Epoch 642/20000 Training Loss: 0.11927051842212677
Epoch 643/20000 Training Loss: 0.12435507774353027
Epoch 644/20000 Training Loss: 0.08754831552505493
Epoch 645/20000 Training Loss: 0.12838169932365417
Epoch 646/20000 Training Loss: 0.1496594250202179
Epoch 647/20000 Training Loss: 0.11192537844181061
Epoch 648/20000 Training Loss: 0.10847268998622894
Epoch 649/20000 Training Loss: 0.11096890270709991
Epoch 650/20000 Training Loss: 0.11157144606113434
Epoch 651/20000 Training Loss: 0.12

...

Epoch 3137/20000 Training Loss: 0.05817835032939911
Epoch 3138/20000 Training Loss: 0.09741011261940002
Epoch 3139/20000 Training Loss: 0.07529369741678238
Epoch 3140/20000 Training Loss: 0.055924512445926666
Epoch 3141/20000 Training Loss: 0.06300758570432663
Epoch 3142/20000 Training Loss: 0.08440344035625458
Epoch 3143/20000 Training Loss: 0.07784033566713333
Epoch 3144/20000 Training Loss: 0.05606796592473984
Epoch 3145/20000 Training Loss: 0.06654930859804153
Epoch 3146/20000 Training Loss: 0.0475783608853817
Epoch 3147/20000 Training Loss: 0.09133085608482361
Epoch 3148/20000 Training Loss: 0.09298364818096161
Epoch 3149/20000 Training Loss: 0.06632695347070694
Epoch 3150/20000 Training Loss: 0.05978942662477493
Epoch 3151/20000 Training Loss: 0.07736018300056458
Epoch 3152/20000 Training Loss: 0.05693785101175308
Epoch 3153/20000 Training Loss: 0.07197047770023346
Epoch 3154/20000 Training Loss: 0.058994218707084656
Epoch 3155/20000 Training Loss: 0.0798799991607666
Epoch 3156/2

Epoch 3294/20000 Training Loss: 0.07834360003471375
Epoch 3295/20000 Training Loss: 0.060788653790950775
Epoch 3296/20000 Training Loss: 0.0884154736995697
Epoch 3297/20000 Training Loss: 0.0839645266532898
Epoch 3298/20000 Training Loss: 0.06538420915603638
Epoch 3299/20000 Training Loss: 0.05391278117895126
Epoch 3300/20000 Training Loss: 0.07726109027862549
Epoch 3300/20000 Validation Loss: 0.04974125325679779
Cleared directory to save new best model.
============>Saving current best model with min_val_loss=0.04974125325679779<=============
Epoch 3301/20000 Training Loss: 0.0648718997836113
Epoch 3302/20000 Training Loss: 0.07167263329029083
Epoch 3303/20000 Training Loss: 0.09131444990634918
Epoch 3304/20000 Training Loss: 0.06988278031349182
Epoch 3305/20000 Training Loss: 0.07338927686214447
Epoch 3306/20000 Training Loss: 0.05490923672914505
Epoch 3307/20000 Training Loss: 0.08390794694423676
Epoch 3308/20000 Training Loss: 0.06469352543354034
Epoch 3309/20000 Training Loss: 0.0

Epoch 3448/20000 Training Loss: 0.059797272086143494
Epoch 3449/20000 Training Loss: 0.07354944199323654
Epoch 3450/20000 Training Loss: 0.06561142951250076
Epoch 3451/20000 Training Loss: 0.06804556399583817
Epoch 3452/20000 Training Loss: 0.09614618122577667
Epoch 3453/20000 Training Loss: 0.0660032331943512
Epoch 3454/20000 Training Loss: 0.0928645208477974
Epoch 3455/20000 Training Loss: 0.08071582019329071
Epoch 3456/20000 Training Loss: 0.08489927649497986
Epoch 3457/20000 Training Loss: 0.047013044357299805
Epoch 3458/20000 Training Loss: 0.07019171863794327
Epoch 3459/20000 Training Loss: 0.05949132889509201
Epoch 3460/20000 Training Loss: 0.06015308201313019
Epoch 3461/20000 Training Loss: 0.06496188789606094
Epoch 3462/20000 Training Loss: 0.06521962583065033
Epoch 3463/20000 Training Loss: 0.10224416851997375
Epoch 3464/20000 Training Loss: 0.07076326757669449
Epoch 3465/20000 Training Loss: 0.06341177225112915
Epoch 3466/20000 Training Loss: 0.07177804410457611
Epoch 3467/2

Epoch 3604/20000 Training Loss: 0.06958694756031036
Epoch 3605/20000 Training Loss: 0.0628698319196701
Epoch 3606/20000 Training Loss: 0.055520251393318176
Epoch 3607/20000 Training Loss: 0.06911913305521011
Epoch 3608/20000 Training Loss: 0.07754819840192795
Epoch 3609/20000 Training Loss: 0.03892862796783447
Epoch 3610/20000 Training Loss: 0.06908397376537323
Epoch 3611/20000 Training Loss: 0.06521175801753998
Epoch 3612/20000 Training Loss: 0.0899176076054573
Epoch 3613/20000 Training Loss: 0.05015973746776581
Epoch 3614/20000 Training Loss: 0.07533019036054611
Epoch 3615/20000 Training Loss: 0.06084330379962921
Epoch 3616/20000 Training Loss: 0.0693037211894989
Epoch 3617/20000 Training Loss: 0.07506394386291504
Epoch 3618/20000 Training Loss: 0.06427001953125
Epoch 3619/20000 Training Loss: 0.07019494473934174
Epoch 3620/20000 Training Loss: 0.08232870697975159
Epoch 3621/20000 Training Loss: 0.06128004193305969
Epoch 3622/20000 Training Loss: 0.06890078634023666
Epoch 3623/20000 

Epoch 3761/20000 Training Loss: 0.0623926967382431
Epoch 3762/20000 Training Loss: 0.09174452722072601
Epoch 3763/20000 Training Loss: 0.09837212413549423
Epoch 3764/20000 Training Loss: 0.07083550840616226
Epoch 3765/20000 Training Loss: 0.04941398650407791
Epoch 3766/20000 Training Loss: 0.07279811799526215
Epoch 3767/20000 Training Loss: 0.05593222379684448
Epoch 3768/20000 Training Loss: 0.07709196954965591
Epoch 3769/20000 Training Loss: 0.060547955334186554
Epoch 3770/20000 Training Loss: 0.08292153477668762
Epoch 3771/20000 Training Loss: 0.077231265604496
Epoch 3772/20000 Training Loss: 0.07166370749473572
Epoch 3773/20000 Training Loss: 0.06200101226568222
Epoch 3774/20000 Training Loss: 0.06350120902061462
Epoch 3775/20000 Training Loss: 0.058579884469509125
Epoch 3776/20000 Training Loss: 0.06242026761174202
Epoch 3777/20000 Training Loss: 0.052989792078733444
Epoch 3778/20000 Training Loss: 0.08255799859762192
Epoch 3779/20000 Training Loss: 0.07636690884828568
Epoch 3780/2

In [ ]:
# dt = datetime.now().strftime("%m%d")
# best_model_fp = f'models/{dt}/model_best_epoch_19200_min_val_loss_0.13920000195503235.pkl'
# with open(best_model_fp, 'rb') as handle:
#     best_model = pickle.load(handle)

# Generating sequences

In [ ]:
word_lst_source, word_lst_recover, word_lst_ref = sampling(best_model, diffusion, tokenizer, data_dir=regular_data_dir, batch_size=10, split='test_custom', seq_len=20)

Generating 20 sentences takes 5 minutes

In [ ]:
word_lst_source

In [ ]:
word_lst_recover

In [ ]:
word_lst_ref